In [363]:
import numpy as np
import pandas as pd

# Load the .npz file
file_path = 'task_markers.npz'  # Replace with your file path
data = np.load(file_path)

# Display the names of the arrays stored in the .npz file
print("Array names in the .npz file:", data.files)

# Accessing each array by name
# for array_name in data.files:
#     print(f"Array name: {array_name}")
#     print(f"Array content:\n{data[array_name]}")

Array names in the .npz file: ['event_ids', 'timestamps']


In [364]:
# data['timestamps']

In [365]:
# data['event_ids']
# data['timestamps']

# Create the DataFrame
df_tasks = pd.DataFrame({
    'event_ids': data['event_ids'][:,0],
    'timestamps': data['timestamps']
})

In [366]:
df_tasks

,event_ids,timestamps
0,-1,83369.837798
1,1,83369.837819
2,2,83372.347878
3,99,83372.347994
4,100,83373.348802
5,3,83373.348817
6,4,83375.863448
7,99,83375.863562
8,100,83376.865493
9,1,83376.865506


In [367]:
# data['buffer']

In [368]:
# len(data['timestamps'])

In [369]:
import pandas as pd
import numpy as np
import os

# Function to process a single .npz file
def process_npz_file(file_path):
    data = np.load(file_path)
    timestamps = data['timestamps']
    buffer = data['buffer']
    
    # Ensure buffer is 2D
    if buffer.ndim == 1:
        buffer = buffer.reshape(-1, 1)
    
    # Combine buffer and timestamps
    combined = np.hstack((buffer, timestamps.reshape(-1, 1)))
    
    # Create DataFrame
    df = pd.DataFrame(combined, columns=[f'buffer_col_{i}' for i in range(buffer.shape[1])] + ['timestamps'])
    
    return df

In [370]:
# data['buffer']

In [371]:
folder_path = 'buffer_data'
dfs = []
index = 1

# Iterate through all .npz files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.npz'):
        file_path = os.path.join(folder_path, file_name)
        df = process_npz_file(file_path)
        df['epoch_number'] = index
        index += 1
        dfs.append(df)

# Concatenate all DataFrames
df_buffers = pd.concat(dfs, ignore_index=True)

# Display the combined DataFrame
print(df_buffers)

      buffer_col_0  buffer_col_1   buffer_col_2  buffer_col_3  buffer_col_4  \
0    -400278.53125 -459527.84375   99575.773438 -392204.03125 -319224.84375   
1    -398722.68750 -458676.50000   98573.703125 -391829.31250 -320129.21875   
2    -397316.75000 -458219.62500   98956.812500 -391431.00000 -320985.00000   
3    -398667.15625 -459112.18750  100076.453125 -391789.96875 -320159.78125   
4    -400213.71875 -459725.53125  100192.773438 -392145.71875 -319299.96875   
...            ...           ...            ...           ...           ...   
2245 -405196.81250 -465760.75000  147542.703125 -397366.56250 -325764.87500   
2246 -403270.56250 -464833.59375  146865.453125 -396856.21875 -326924.93750   
2247 -402844.43750 -464932.59375  147759.703125 -396731.15625 -327175.03125   
2248 -404691.93750 -465929.09375  148458.140625 -397192.93750 -326078.53125   
2249 -405715.56250 -466215.56250  148320.734375 -397432.37500 -325520.18750   

       buffer_col_5  buffer_col_6  buffer_col_7  bu

In [372]:
df_buffers = df_buffers.sort_values('timestamps').reset_index(drop=True)
df_tasks = df_tasks.sort_values('timestamps').reset_index(drop=True)

In [373]:
# Perform an asof merge to find the closest earlier and later event_id
df_buffers['prev_event_id'] = pd.merge_asof(df_buffers, df_tasks,
                                            left_on='timestamps', right_on='timestamps', direction='backward')['event_ids']

df_buffers['next_event_id'] = pd.merge_asof(df_buffers, df_tasks,
                                            left_on='timestamps', right_on='timestamps', direction='forward')['event_ids']

In [374]:
df_tasks

,event_ids,timestamps
0,-1,83369.837798
1,1,83369.837819
2,2,83372.347878
3,99,83372.347994
4,100,83373.348802
5,3,83373.348817
6,4,83375.863448
7,99,83375.863562
8,100,83376.865493
9,1,83376.865506


In [375]:
df_buffers[df_buffers['timestamps'] > 78348.486256]
# df_tasks

,buffer_col_0,buffer_col_1,buffer_col_2,buffer_col_3,buffer_col_4,buffer_col_5,buffer_col_6,buffer_col_7,buffer_col_8,buffer_col_9,...,buffer_col_11,buffer_col_12,buffer_col_13,buffer_col_14,buffer_col_15,buffer_col_16,timestamps,epoch_number,prev_event_id,next_event_id
0,-400278.53125,-459527.84375,99575.773438,-392204.03125,-319224.84375,99663.304688,-401490.53125,-750000.125,0.293213,0.929199,...,-2.593994,-0.061035,0.915527,33.333336,235578.0,1.0,83369.328664,1,NaN,-1
1,-398722.68750,-458676.50000,98573.703125,-391829.31250,-320129.21875,98789.531250,-402223.40625,-750000.125,0.292236,0.929443,...,-2.624511,-0.152588,0.793457,33.333336,235579.0,1.0,83369.328681,1,NaN,-1
2,-397316.75000,-458219.62500,98956.812500,-391431.00000,-320985.00000,99124.898438,-402811.53125,-750000.125,0.292969,0.929199,...,-2.563476,-0.091553,0.793457,33.333336,235580.0,1.0,83369.328684,1,NaN,-1
3,-398667.15625,-459112.18750,100076.453125,-391789.96875,-320159.78125,100089.867188,-402117.75000,-750000.125,0.293213,0.930176,...,-2.655029,-0.061035,0.640869,33.333336,235581.0,1.0,83369.331840,1,NaN,-1
4,-400213.71875,-459725.53125,100192.773438,-392145.71875,-319299.96875,100165.593750,-401474.81250,-750000.125,0.293701,0.928467,...,-2.563476,0.061035,0.610352,33.333336,235582.0,1.0,83369.331845,1,NaN,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2245,-405196.81250,-465760.75000,147542.703125,-397366.56250,-325764.87500,147687.281250,-407703.62500,-750000.125,0.290527,0.930176,...,-2.502441,0.000000,0.579834,33.333336,239840.0,1.0,83386.345058,9,1.0,2
2246,-403270.56250,-464833.59375,146865.453125,-396856.21875,-326924.93750,147079.406250,-408589.37500,-750000.125,0.290771,0.929932,...,-2.502441,0.030518,0.518799,33.333336,239841.0,1.0,83386.351281,9,1.0,2
2247,-402844.43750,-464932.59375,147759.703125,-396731.15625,-327175.03125,147879.234375,-408700.68750,-750000.125,0.289795,0.930664,...,-2.502441,0.122070,0.579834,33.333336,239842.0,1.0,83386.358323,9,1.0,2
2248,-404691.93750,-465929.09375,148458.140625,-397192.93750,-326078.53125,148464.578125,-407827.43750,-750000.125,0.291260,0.929688,...,-2.502441,-0.061035,0.640869,33.333336,239843.0,1.0,83386.358344,9,1.0,2


In [376]:
markers = {
        'forward_start': [1],
        'forward_end': [2],
        'reverse_start': [3],
        'reverse_end': [4],
        'rest_start': [99],
        'rest_end': [100],
        'task_start': [-1],
        'task_end': [-2]
    }

In [377]:
# Create a function to determine the phase
def determine_phase(row):
    if row['prev_event_id'] in markers['forward_start'] and row['next_event_id'] in markers['forward_end']:
        return 'forward'
    elif row['prev_event_id'] in markers['reverse_start'] and row['next_event_id'] in markers['reverse_end']:
        return 'reverse'
    elif row['prev_event_id'] in markers['rest_start'] and row['next_event_id'] in markers['rest_end']:
        return 'rest'
    else:
        return 'unknown'

In [378]:
# Apply the function to each row in df_buffers
df_buffers['phase'] = df_buffers.apply(determine_phase, axis=1)

In [379]:
# Display the DataFrame with phases
# df_buffers[df_buffers['phase'] == 'forward']

In [380]:
# df_buffers['phase'].unique()

In [381]:
grouped = df_buffers.groupby(['epoch_number', 'phase']).size().reset_index(name='count')

In [382]:
# df_buffers.groupby('epoch_number').size().reset_index(name='count')

In [383]:
valid_phases = ['forward', 'reverse', 'rest']
filtered_groups = grouped[(grouped['phase'].isin(valid_phases)) & (grouped['count'] == 250)]

In [384]:
# filtered_groups

In [385]:
mask = df_buffers.set_index(['epoch_number', 'phase']).index.isin(filtered_groups.set_index(['epoch_number', 'phase']).index)
filtered_df_buffers = df_buffers[mask].reset_index(drop=True)


In [386]:
filtered_df_buffers

,buffer_col_0,buffer_col_1,buffer_col_2,buffer_col_3,buffer_col_4,buffer_col_5,buffer_col_6,buffer_col_7,buffer_col_8,buffer_col_9,...,buffer_col_12,buffer_col_13,buffer_col_14,buffer_col_15,buffer_col_16,timestamps,epoch_number,prev_event_id,next_event_id,phase
0,-399026.59375,-459300.03125,105389.015625,-392355.37500,-321078.71875,105613.609375,-403083.87500,-750000.125,0.291748,0.929443,...,0.061035,0.671387,33.333336,236080.0,1.0,83371.329221,2,1.0,2,forward
1,-397982.93750,-459059.62500,106054.023438,-392057.12500,-321710.56250,106211.382812,-403493.00000,-750000.125,0.292969,0.930420,...,0.030518,0.671387,33.333336,236081.0,1.0,83371.329223,2,1.0,2,forward
2,-399598.34375,-460030.31250,107110.812500,-392477.68750,-320736.71875,107115.554688,-402696.37500,-750000.125,0.293457,0.931152,...,-0.030518,0.671387,33.333336,236082.0,1.0,83371.331689,2,1.0,2,forward
3,-400947.31250,-460510.87500,107113.765625,-392785.15625,-319992.40625,107101.875000,-402153.21875,-750000.125,0.293945,0.930420,...,0.030518,0.579834,33.333336,236083.0,1.0,83371.331698,2,1.0,2,forward
4,-400793.00000,-460221.37500,106402.351562,-392801.25000,-320044.90625,106535.843750,-402270.81250,-750000.125,0.293213,0.929199,...,-0.061035,0.488281,33.333336,236084.0,1.0,83371.340000,2,1.0,2,forward
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-405196.81250,-465760.75000,147542.703125,-397366.56250,-325764.87500,147687.281250,-407703.62500,-750000.125,0.290527,0.930176,...,0.000000,0.579834,33.333336,239840.0,1.0,83386.345058,9,1.0,2,forward
996,-403270.56250,-464833.59375,146865.453125,-396856.21875,-326924.93750,147079.406250,-408589.37500,-750000.125,0.290771,0.929932,...,0.030518,0.518799,33.333336,239841.0,1.0,83386.351281,9,1.0,2,forward
997,-402844.43750,-464932.59375,147759.703125,-396731.15625,-327175.03125,147879.234375,-408700.68750,-750000.125,0.289795,0.930664,...,0.122070,0.579834,33.333336,239842.0,1.0,83386.358323,9,1.0,2,forward
998,-404691.93750,-465929.09375,148458.140625,-397192.93750,-326078.53125,148464.578125,-407827.43750,-750000.125,0.291260,0.929688,...,-0.061035,0.640869,33.333336,239843.0,1.0,83386.358344,9,1.0,2,forward


In [387]:
filtered_df_buffers['epoch_number'].unique()

array([2, 5, 7, 9], dtype=int64)

In [388]:
columns_to_select = df_buffers.columns[:8].tolist() + ['epoch_number', 'timestamps','phase']
final_df_buffers = filtered_df_buffers[columns_to_select]

In [389]:
channel_names = ["Fz", "C3", "Cz", "C4", "Pz", "PO7", "Oz", "PO8"]

rename_dict = {old_name: new_name for old_name, new_name in zip(final_df_buffers.columns[:8], channel_names)}
final_df_buffers = final_df_buffers.rename(columns=rename_dict)

In [390]:
final_df_buffers.groupby(['epoch_number', 'phase']).size().reset_index(name='count')

,epoch_number,phase,count
0,2,forward,250
1,5,forward,250
2,7,reverse,250
3,9,forward,250


In [410]:
epochs_data = []
events = []

# Need to add rest:3, I have remove it temporarily as when I record for test I am not getting pure rest (250 s) signals.
# but if we run the psychopy iterations, we will have representation of rest state

event_id = {'forward': 1, 'reverse': 2}
phases = ['forward', 'reverse', 'rest']

In [411]:
for epoch_number, phase in final_df_buffers.groupby(['epoch_number', 'phase']):
    phase_name = epoch_number[1]
    # if phase_name not in phases:
    #     continue
    epoch_data = phase[channel_names].values.T  # Transpose to get (n_channels, n_times)
    epochs_data.append(epoch_data)
    events.append([len(epochs_data) - 1, 0, event_id[phase_name]])

In [412]:
epochs_data = np.array(epochs_data)  # Shape should be (n_epochs, n_channels, n_times)
events = np.array(events)  # Shape should be (n_epochs, 3)

In [413]:
import mne
from mne import create_info
from mne.epochs import EpochsArray

sfreq = 125

In [414]:
# Step 4: Create MNE info structure
info = create_info(ch_names=channel_names, sfreq=sfreq, ch_types='eeg')

# Step 5: Create MNE Epochs object
epochs = EpochsArray(epochs_data, info, events, event_id=event_id, tmin=0)

Not setting metadata
4 matching events found
No baseline correction applied
0 projection items activated


In [415]:
# Example of how to feed the epochs into a classification model for training
# This is just a placeholder example; replace with actual model and training code
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from mne.decoding import Vectorizer

X = epochs.get_data()  # Shape: (n_epochs, n_channels, n_times)
y = epochs.events[:, 2]  # Shape: (n_epochs,)

# Reshape the data to (n_samples, n_features)
X = X.reshape(len(X), -1)

# Step 7: Create and train a classifier
clf = make_pipeline(Vectorizer(), RandomForestClassifier(n_estimators=100))
clf.fit(X, y)

C:\Users\iamth\AppData\Local\Temp\ipykernel_16324\2915869972.py:7: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X = epochs.get_data()  # Shape: (n_epochs, n_channels, n_times)


Pipeline(steps=[('vectorizer',
                 <mne.decoding.transformer.Vectorizer object at 0x0000015934652790>),
                ('randomforestclassifier', RandomForestClassifier())])

In [416]:
test_index = 0  # Use the first training data point for prediction
test_sample = X[test_index].reshape(1, -1)  # Reshape to (1, n_features)
predicted_label = clf.predict(test_sample)
actual_label = y[test_index]
print(f"Predicted label: {predicted_label[0]}, Actual label: {actual_label}")

Predicted label: 1, Actual label: 1
